# Overview
This notebook is designed to demonstrate how the attached SQL database was constructed. Weekly Shonen Jump is a weekly manga (comic book) magazine published in Japan which includes roughly 20 regularly serialized series per issue. The database `shonen_jump.sqlite3` records series published in the magazine and tracks when series perform well (and are placed higher in the magazine's table of contents), receive color pages, or grace the cover of the magazine.

In [1]:
import sqlite3
connection = sqlite3.connect("shonen_jump.sqlite3")
cursor=connection.cursor()

# We will enable foreign keys
cursor.execute("PRAGMA foreign_keys = ON")

First we will create a table to hold the possible statuses of a manga: "Ongoing", "Complete", "Hiatus".

In [2]:
cursor.execute("CREATE TABLE IF NOT EXISTS statuses(status TEXT PRIMARY KEY)")
cursor.execute("""
               INSERT OR IGNORE INTO statuses VALUES
               ('Ongoing'),
               ('Complete'),
               ('Hiatus'),
               ('Transferred')""")
connection.commit()

Similarly we will create a table of major genres.

In [3]:
cursor.execute("CREATE TABLE IF NOT EXISTS genres(genre TEXT PRIMARY KEY)")
cursor.execute("""
               INSERT OR IGNORE INTO genres VALUES
               ('Battle'),
               ('Sports'),
               ('Romance'),
               ('Comedy'),
               ('Other')""")
connection.commit()

Next we will create a table to record which series are running (or have run) in the magazine.

In [4]:
cursor.execute("""CREATE TABLE IF NOT EXISTS series(
               title TEXT PRIMARY KEY,
               writer TEXT,
               artist TEXT,
               total_chapters INTEGER CHECK (total_chapters > 0),
               genre TEXT,
               status TEXT,
               start_date TEXT CHECK (start_date>'1900'),
               end_date TEXT CHECK (end_date >= start_date),
               FOREIGN KEY(status) REFERENCES statuses(status),
               FOREIGN KEY(genre) REFERENCES genres(genre)
               )""")

To add multiple series at once to this new table we will use the execute many command. First we collect information on the series.

In [5]:
# Series will be added in the form (title, chapters, writer, artist, start, genre)

series_to_add=[
    ('One Piece', 1156,'Eiichiro Oda','Eiichiro Oda', 'Battle'),
    ('Kagurabachi',89, 'Takeru Hokazono','Takeru Hokazono', 'Battle'),
    ('Sakamoto Days',224, 'Yuto Suzuki','Yuto Suzuki', 'Battle'),
    ('Ichi the Witch',45, 'Osamu Nishi','Shiro Usazaki', 'Battle'),
    ('Kaedegami',7, 'Jun Harukawa','Jun Harukawa', 'Battle'),
    ('The Elusive Samurai',213,'Yusei Matsui','Yusei Matsui', 'Other'),
    ('Shinobi Undercover',44,'Ippon Takegushi','Santa Mitarashi', 'Battle'),
    ("Nue's Exorcist",109,'Kota Kawae','Kota Kawae', 'Battle'),
    ('Ultimate Exorcist Kiyoshi', 55,'Shoichi Usui','Shoichi Usui', 'Battle'),
    ('Hima-Ten!',53,'Genki Ono','Genki Ono', 'Romance'),
    ('Witch Watch',212,'Kenta Shinohara','Kenta Shinohara', 'Comedy'),
    ('Akane-banashi',169,'Yuki Suenaga','Takamasa Moue', 'Other'),
    ('Blue Box',206,'Kouji Miura','Kouji Miura', 'Romance'),
    ('Ekiden Bros',6,'Daiki Nono','Daiki Nono', 'Sports'),
    ('Harukaze Mound', 8, 'Togo Goto', 'Kento Matsuura',  'Sports'),
    ('Kill Blue', 112, 'Tadatoshi Fujimaki','Tadatoshi Fujimaki','Comedy'),
    ('Me & Roboco',244,'Shuhei Miyazaki','Shuhei Miyazaki','Comedy'),
    ('Nice Prison',15,'Tatsuya Suganuma','Tatsuya Suganuma','Comedy'),
    ('Otr of the Flame',13,'Yuki Kawaguchi','Yuki Kawaguchi','Battle'),
    ('Ping-Pong Peril',5,'Yoshiharu Kataoka','Yoshiharu Kataoka','Sports')
]

In [6]:
cursor.executemany("""
               INSERT OR IGNORE INTO series (title, total_chapters, writer, artist, genre, status) 
                   VALUES (?,?,?,?,?,'Ongoing')""",series_to_add)
connection.commit()

Now we will turn our attention to creating a table listing individual chapters appearing in the magazine. First we will list the kinds of chapters possible.

In [7]:
cursor.execute("CREATE TABLE IF NOT EXISTS chapter_types(type TEXT PRIMARY KEY DEFAULT 'Normal')")
cursor.execute("""
               INSERT OR IGNORE INTO chapter_types VALUES
               ('Normal'),
               ('Color'),
               ('Cover'),
               ('Absent')""")
connection.commit()

In [8]:
cursor.execute("""CREATE TABLE IF NOT EXISTS chapters(
               series TEXT NOT NULL,
               release_date TEXT NOT NULL,
               toc_rank INTEGER,
               chapter INTEGER,
               type TEXT,
               placement INTEGER,
               Primary KEY (series, release_date),
               FOREIGN KEY(series) REFERENCES series(title)
               )""")

We also want the total chapter count in the series table to increment by 1 whenever a new chapter is published.  For this we will create two triggers for when the chapters table is acted upon.  One for insertions and one for updates.

In [9]:
cursor.execute("""
               CREATE TRIGGER IF NOT EXISTS update_chapter_on_insert INSERT ON chapters
               WHEN NEW.chapter > (SELECT total_chapters FROM series WHERE title = NEW.series)
               BEGIN
               UPDATE series SET total_chapters = NEW.chapter WHERE title = NEW.series;
               END;
""")

cursor.execute("""
               CREATE TRIGGER IF NOT EXISTS update_chapter_on_update UPDATE OF chapter ON chapters
               WHEN NEW.chapter > (SELECT total_chapters FROM series WHERE title = NEW.series)
               BEGIN
               UPDATE series SET total_chapters = NEW.chapter WHERE title = NEW.series;
               END;
""")
connection.commit()

To fill in the chapters table we will use the `add_week` function created in the script `sj_db_functions.py` which simplifies the process of adding each week's worth of chapters. The function is repeated below for completeness. 

In [10]:
def add_week(date: str, data, recency=None):

    """This function updates the Shonen Jump database (shonen_jump.sqlite3) by adding a week's worth of chapters to the 
    chapters table.  Date should be a string of the form YYYY-MM-DD. 
    
    By using the optional parameter "recency" chapter numbers can
    be automatically assigned based on the currently known chapters. If recency='latest' then the chapter number will be 1 more than
    the currently recorded total chapters of the series. If recency='previous' then the chapter number will be 1 less than the smallest recorded chapter
    for the series in the database. 
    
    Data should be a list of dictionaries, with each dictionary containing the following:
    
        {'s': "Name of Series as a string"  # Required

        'r': Rank in the table of contents as an int. # Optional. Should be omitted if chapter type is not 'Normal'

        't': "Type of chapter as a string: 'Absent','Color', 'Cover', or 'Normal'" # Optional, defaults to normal.

        'c': Chapter number as an int. # Optional. Will be overwritten if the recency parameter of add_week is included.}"""
    
    # First we scan the user entered data to warn for errors.
    for entry in data:
        if entry.get('s')==None:
            raise Exception('The name of each series must be included.')
        if entry.get('r')!=None and entry.get('t').lower() in ['absent','color', 'cover']:
            raise Exception('A special chapter cannon be given a ranking in the table of contents.')
        if entry.get('r')<1:
            raise Exception('Rankings cannot be less than 1.')
        if entry.get('c')!=None and recency in ['latest', 'previous']:
            raise Warning('Chapter numbers which were provided have been ignored because the recency parameter was set.')
        entry['d']=date
        
    # We now connect to the database
    connection = sqlite3.connect("shonen_jump.sqlite3")
    cursor=connection.cursor()

    # If recency is set to latest we autocalculate the chapter numbers.
    if recency=='latest':
        cursor.executemany("""
                           INSERT INTO chapters (series, release_date, number, toc_rank, type)
                           VALUES (:s, :d,
                           CASE WHEN :t='Absent' THEN NULL
                           ELSE (SELECT 1 + total_chapters FROM series WHERE title=:s),:r,:t)""",data)
    elif recency=='previous':
        cursor.executemany("""
                           INSERT INTO chapters (series, release_date, number, toc_rank, type)
                           VALUES (:s, :d,
                           CASE WHEN :t='Absent' THEN NULL
                           ELSE (SELECT -1 + MIN(number) FROM chapters WHERE series=:s),:r,:t)""",data)
    # Otherwise we add the information given as is.
    else:
        cursor.executemany("""
                           INSERT INTO chapters (series, release_date, number, toc_rank, type)
                           VALUES (:s,:d,:c,:r,:t)""",data)
    connection.commit()
    connection.close()

In [11]:
connection.close()